In [21]:
from utils import *
from szloc import *
from szloc_loss import *
from szloc_loader import *
import numpy as np 
import torch
import os
import json
import pandas as pd
from visualization import *
from szloc_loss import *
from szloc_train import *

In [2]:
device = 'cpu'
data_root = '/home/deeksha/EEG_Sz/miccai23/'
manifest = read_manifest('/home/deeksha/EEG_Sz/miccai23/'+ 'data/tuh_single_windowed_manifest.csv', ',')


n=5
use_cuda = False
cvfold = 0
foldroot = 'final_models/fold'+str(cvfold)+'/'
allfiles = os.listdir(foldroot)
testfile = list(filter(lambda f:f.startswith('pts_test'), allfiles))[0]
val_pts = np.load(foldroot+testfile)
val_set =  szlocLoader(data_root, val_pts, manifest, addNoise=False, 
                                        input_mask=None, ablate=False, permute=False, normalize=True)
validation_loader = DataLoader(val_set, batch_size=1, shuffle=True)
                

In [3]:
chn_neighbours = {0: [1,2,3,4], 
                  1: [0,4,5,6], 
                  2: [0,3,4,7,8], 
                  3: [0,2,4,8,9], 
                  4: [0,1,3,5,9], 
                  5: [1,4,6,9,10],
                  6: [1,4,5,10,11], 
                  7: [2,8,12,13, 17], 
                  8: [2,3,4,7,9,12,13,14], 
                  9: [3,4,5,8,10,13,14,15], 
                 10: [4,5,6,9,11,14,15,16], 
                 11: [6, 10, 15, 16, 18], 
                 12: [7, 8, 13, 17], 
                 13: [7, 8, 9, 12, 14, 17],
                 14: [8,9,10,13,15,17,18],
                 15: [9,10,11,14,16,18], 
                 16: [10,11,15,18], 
                 17: [7,12,13,14,18], 
                 18: [11, 14,15, 16, 17]}

def check_neighborhood(max_chn, onset_map):
    check = False
    for i in range(19):
        if onset_map[i]==1:
            surrounding = chn_neighbours[i]
            if max_chn in surrounding:
                check = True
    return check

def final_loc(psoz, true_onset):
            n = psoz.shape[0]
            m = psoz.max(1).reshape(n,1)
            psoz = psoz/m
            ysoz = psoz.mean(0)
            #ysoz /= ysoz.max()
            max_chn_loc = np.argmax(ysoz)                    
            max_chn_correct = 1 if true_onset[max_chn_loc]==1 else 0
            
            
            if check_neighborhood(max_chn_loc, true_onset) and true_onset.sum()<=2:
                        max_chn_correct =1 
                       
 
  
            Uev = psoz.var(0)

            return ysoz, Uev, max_chn_correct

In [4]:
ablate = {'szloc':{'szcorr': [], 'ptcorr':[], 'szunc': [],'ptunc': []}}
modelname = 'szloc'

In [ ]:
for cvfold in range(0,15,1):
    foldroot = 'final_models/fold'+str(cvfold)+'/'
    allfiles = os.listdir(foldroot)
    #modelfiles = list(filter(lambda f:f.endswith('.tar'), allfiles))
    
    modelfile = list(filter(lambda f:f.startswith('szloc'), allfiles))[0]
    testfile = list(filter(lambda f:f.startswith('pts_test'), allfiles))[0]
    
    val_pts = np.load(foldroot+testfile)
    model = ctg_11_8(transformer_dropout = 0.15, cnn_dropout=0.15)
    model.to(device)
            
    statedict = torch.load(foldroot+modelfile)
  
    
    model.load_state_dict(statedict)
    model.double()
    corr_pt = 0
    uncs = []
    
    
    for pt in val_pts:
            
            sets =  szlocLoader(data_root, [pt], manifest, addNoise=False, normalize=True, 
                                   input_mask=None, ablate=False, permute=False)
            loader = DataLoader(sets, batch_size=1, shuffle=True)
            
            
            correct_loc = 0
            tot_sz = 0
            ysoz_all = []
            a_all = []
           
            for idx, data in enumerate(loader):
                    
                    inputs = data['buffers'].double()
                    B,Nsz,T, C, N = inputs.shape
                    chn_pos = torch.arange(C)
                    true_onset = data['onset map'].detach().numpy().reshape(-1)
                    chn_sz_pred, sz_pred, attn_onset_map, psoz = model(inputs)


                    ysoz_all.append(psoz.reshape(Nsz, C).detach().cpu().numpy())
                   
                    tot_sz += Nsz
                    
            
            #f, ax = plt.subplots(1,1, figsize = (4, 4))
            #ax[0].plot(sz_pred[:, :, 1].detach().numpy().reshape(-1))
            ysoz_all = np.concatenate(ysoz_all).reshape(tot_sz, 19)
            psoz = ysoz_all#[:10, :]
            ysoz, conf_y, max_chn_correct_y = final_loc(psoz, true_onset)

            
            corr_pt += max_chn_correct_y
            
            uncs.append(conf_y)
            #pos2d = topoplot(ysoz, ax,
            #                   title=pt, zone=1,
            #                    lobe_correct=max_chn_correct_y,
            #                    lat_correct=max_chn_correct_y,
            #                    onset_map=true_onset)
            #plt.show()
    
    print(modelname, cvfold, corr_pt, np.array(uncs).mean() )
    del model
        
    ablate[modelname]['ptcorr'].append(corr_pt)
    ablate[modelname]['ptunc'].append(np.array(uncs).max(1).mean())
        
    print('\n')

In [ ]:
for cvfold in range(0,15,1):
    foldroot = 'final_models/fold'+str(cvfold)+'/'
    allfiles = os.listdir(foldroot)
    
    modelfile = list(filter(lambda f:f.startswith('szloc'), allfiles))[0]
    testfile = list(filter(lambda f:f.startswith('pts_test'), allfiles))[0]
    val_pts = np.load(foldroot+testfile)
    sets =  szlocLoader(data_root, val_pts, manifest, addNoise=False, normalize=True, 
                                   input_mask=None, ablate=False, permute=False)
    loader = DataLoader(sets, batch_size=1, shuffle=True)
    
    model = ctg_11_8(transformer_dropout = 0.15, cnn_dropout=0.15)
    model.to(device)
            
    statedict = torch.load(foldroot+'/szloc_final/'+modelfile)
  
    
    model.load_state_dict(statedict)
    model.double()
    corr_sz = []
    mc_var = []
        
    for idx, data in enumerate(loader):

            ysoz_all = []
            #print(idx)
            
            inputs = data['buffers'].long()
            
            B,Nsz,T, C, N = inputs.shape
            chn_pos = torch.arange(C)
            true_onset = data['onset map'].detach().numpy().reshape(-1)
            #tot_sz += Nsz
            for mcmc in range(20):   

                    
                    inputs = data['buffers'].double()
                    B,Nsz,T, C, N = inputs.shape
                    chn_pos = torch.arange(C)
                    true_onset = data['onset map'].detach().numpy().reshape(-1)
                    with torch.no_grad():
                         chn_sz_pred, sz_pred, attn_onset_map, psoz = model(inputs)


                    ysoz_all.append(psoz.reshape(Nsz, C).detach().cpu().numpy())
            ysoz_all = np.concatenate(ysoz_all).reshape(20, -1, 19)
            #print(ysoz_all.shape)
            for szn in range(Nsz):
                
                    ysoz, conf_y, max_chn_correct_y = final_loc(ysoz_all[:, szn, :].reshape(-1, 19), true_onset)
                    corr_sz.append(max_chn_correct_y)
                
                    mc_var.append(conf_y)

            #f, ax = plt.subplots(1,10, figsize = (20,2))
            '''
            ysoz, conf_y, max_chn_correct_y = final_loc(psoz[j,:,].reshape(1,-1),
                                                            true_onset)
            pos2d = topoplot(ysoz, ax[j],
                                title='ysoz', zone=1,
                                lobe_correct=max_chn_correct_y,
                                lat_correct=max_chn_correct_y,
                                onset_map=true_onset)
            plt.show()
            #print(Nsz, data['patient numbers'])
            '''
            del ysoz_all
    del model
        
    print(modelname, cvfold, np.array(corr_sz).mean())
        
    ablate[modelname]['szcorr'].append(np.array(corr_sz).mean())
    ablate[modelname]['szunc'].append(np.array(mc_var).mean())
    del corr_sz, mc_var
    del loader
           

In [27]:
model = ctg_11_8()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
sum([np.prod(p.size()) for p in model_parameters])

490791